<a href="https://colab.research.google.com/github/deveshtripath/plotly/blob/main/Copy_of_task_27_solution_devesh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import requests
from io import StringIO

### Problem-1:

You are given a SQL file link: https://drive.google.com/file/d/1WFt7B84LTHhMueoKmz8W-PRo7xXqmZf3/view?usp=share_link. Read the data by using the file and store it in a excel file. In this data, there are 3 tables named "invoices", "order_leads" and "sales_sql". So create 3 sheets to your excel file.

In [6]:
!pip install mysql.connector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 65.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp39-cp39-linux_x86_64.whl size=247964 sha256=d8e81451878bca0df14f7e2ef4106bde243e42417b1669fbc08b978db40d0e90
  Stored in directory: /root/.cache/pip/wheels/7b/14/39/5aad423666e827dfe9a1fbcd111ac17171e7c9865d570780ce
Successfully built mysql.connector


In [7]:
# code here
import mysql.connector
import pandas as pd

conn = mysql.connector.connect(host="localhost",user="root",password="",database="temp_db")

invoices = pd.read_sql_query('SELECT * FROM invoices',conn)
order_lead = pd.read_sql_query('SELECT * FROM order_leads',conn)
sales_sql = pd.read_sql_query('SELECT * FROM sales_sql',conn)


with pd.ExcelWriter('supermarket.xlsx') as writer:
    invoices.to_excel(writer,sheet_name="invoices")
    order_lead.to_excel(writer,sheet_name="order_lead")
    sales_sql.to_excel(writer,sheet_name="sales_sql")

### Problem-2

Go to the site: https://rapidapi.com/wirefreethought/api/geodb-cities. From here, you have to grab the API and have to choose proper routes to get the cities of different countries. After getting the right API, hit that API and create a dataframe of all the cities that you can get by using the API. Then store the dataframe to a SQL. If you need to create an account or have to subscribe, then do that (it has free subscription but has some limitations. Use that free subscription and modify your accordingly to get all the data).  

In [1]:
### code here
### Rapid API 
import requests

url = "https://wft-geo-db.p.rapidapi.com/v1/geo/adminDivisions"

headers = {
    "X-RapidAPI-Key": "b0f75c828fmsh2b0480a62ab8d23p18178ajsn3267e033ae6a",
    "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

data = response.json()['data']
links = response.json()['links']
metadata = response.json()['metadata']

In [ ]:
next_url = links[1]['href']

In [ ]:
count = metadata['totalCount']//5

In [ ]:
import time


for i in range(1,count):
    
    num_iters += 1
    
    url = "https://wft-geo-db.p.rapidapi.com"
    page = url + next_url
    
    response = requests.request("GET", page, headers=headers)
    
    print(response.json())
    
    links = response.json()['links']
    
    for item in links:
        if item['rel'] == 'next':
            next_url = item['href']
            
    data.extend(response.json()['data'])
    
    time.sleep(2)
    

In [ ]:
cities = pd.DataFrame(data)

In [ ]:
import pymysql

from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/temp_db")

cities.to_sql("cities",con=engine,if_exists='append')

### Problem 3:

Go to this url: https://www.flipkart.com/search?q=smartphones. This is the url to find phones in flipkart website. You have to extract the below things:
1. image url of the phone
2. name of the image
3. average ratings
4. total ratings
5. total reviews
6. discounted price
7. actual price

Extract all the phones which are available in this website. So you have to use the pagination concept. **Also after requesting every page through the url, please wait for a while (minimum 2-3 seconds), otherwise your IP address can be banned to access the flipkart website later.**

After collecting all the data, save that in a JSON file.

In [2]:
# code here

# code here
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np

In [3]:
webpage = requests.get('https://www.flipkart.com/search?q=smartphones&page=1').text

In [4]:
soup = BeautifulSoup(webpage,'lxml')

In [5]:
all_divs = soup.find_all('div', class_="_2kHMtA")

In [6]:
phone_image_url = []
phone_name = []
phone_rating = []
phone_num_ratings = []
phone_price = []

for phone in all_divs:
  try:
    phone_image_url.append(phone.find("img",class_="_396cs4")['src'])
  except:
    phone_image_url.append(np.nan)

  try:
    phone_name.append(phone.find("div",class_="_4rR01T").text.strip())
  except:
    phone_name.append(np.nan)

  try:
    phone_rating.append(phone.find("div",class_="_3LWZlK").text.strip())
  except:
    phone_rating.append(np.nan)

  try:
    phone_num_ratings.append(phone.find_all('span',class_='_2_R_DZ')[0].find_all('span')[0].find_all('span')[0].text.strip())
  except:
    phone_num_ratings.append(np.nan)

  try:
    phone_price.append(phone.find("div",class_="_30jeq3 _1_WHN1").text.strip())
  except:
    phone_price.append(np.nan)


df = pd.DataFrame({'phone_name':phone_name,'phone_price':phone_price,'phone_rating':phone_rating,'phone_num_rating':phone_num_ratings,'phone_img_url':phone_image_url})
df

,phone_name,phone_price,phone_rating,phone_num_rating,phone_img_url
0,"realme C30 (Denim Black, 32 GB)","₹5,749",4.3,"93,635 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
1,"realme C30 (Denim Black, 32 GB)","₹6,749",4.1,"43,575 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
2,"realme C30 (Lake Blue, 32 GB)","₹5,749",4.3,"93,635 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
3,"Infinix Smart 6 HD (Force Black, 32 GB)","₹5,999",4.3,"27,023 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
4,"realme C30 (Bamboo Green, 32 GB)","₹6,749",4.1,"43,575 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
5,"realme C30 (Lake Blue, 32 GB)","₹6,749",4.1,"43,575 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
6,"realme C30 (Bamboo Green, 32 GB)","₹5,749",4.3,"93,635 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
7,"Infinix NOTE 12i (Alpine White, 64 GB)","₹10,499",4.3,"3,034 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
8,"SAMSUNG Galaxy F13 (Waterfall Blue, 64 GB)","₹10,999",4.4,"1,32,390 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
9,"POCO M4 5G (Cool Blue, 64 GB)","₹10,999",4.2,"61,024 Ratings",https://rukminim1.flixcart.com/image/312/312/l...


In [10]:
final_df = pd.DataFrame()

for i in range(1,45):

  webpage = requests.get('https://www.flipkart.com/search?q=smartphones&page={}'.format(i)).text
  soup = BeautifulSoup(webpage,'lxml')
  all_divs = soup.find_all('div', class_="_2kHMtA")

  phone_image_url = []
  phone_name = []
  phone_rating = []
  phone_num_ratings = []
  phone_price = []

  for phone in all_divs:
    try:
      phone_image_url.append(phone.find("img",class_="_396cs4")['src'])
    except:
      phone_image_url.append(np.nan)

    try:
      phone_name.append(phone.find("div",class_="_4rR01T").text.strip())
    except:
      phone_name.append(np.nan)

    try:
      phone_rating.append(phone.find("div",class_="_3LWZlK").text.strip())
    except:
      phone_rating.append(np.nan)

    try:
      phone_num_ratings.append(phone.find_all('span',class_='_2_R_DZ')[0].find_all('span')[0].find_all('span')[0].text.strip())
    except:
      phone_num_ratings.append(np.nan)

    try:
      phone_price.append(phone.find("div",class_="_30jeq3 _1_WHN1").text.strip())
    except:
      phone_price.append(np.nan)


  df = pd.DataFrame({'phone_name':phone_name,'phone_price':phone_price,'phone_rating':phone_rating,'phone_num_rating':phone_num_ratings,'phone_img_url':phone_image_url})
  final_df = final_df.append(df)

  phone_image_url.clear()
  phone_name.clear()
  phone_rating.clear()
  phone_num_ratings.clear()
  phone_price.clear()

  time.sleep(3)


<ipython-input-10-cee5662e486a>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
<ipython-input-10-cee5662e486a>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
<ipython-input-10-cee5662e486a>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
<ipython-input-10-cee5662e486a>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(df)
<ipython-input-10-cee5662e486a>:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.a

In [12]:
len(final_df)

984

In [13]:
final_df

,phone_name,phone_price,phone_rating,phone_num_rating,phone_img_url
0,"realme C30 (Denim Black, 32 GB)","₹5,749",4.3,"93,635 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
1,"realme C30 (Denim Black, 32 GB)","₹6,749",4.1,"43,575 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
2,"realme C30 (Lake Blue, 32 GB)","₹5,749",4.3,"93,635 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
3,"Infinix Smart 6 HD (Force Black, 32 GB)","₹5,999",4.3,"27,023 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
4,"realme C30 (Bamboo Green, 32 GB)","₹6,749",4.1,"43,575 Ratings",https://rukminim1.flixcart.com/image/312/312/l...
...,...,...,...,...,...
19,"Infinix HOT 20 Play (Aurora Green, 64 GB)","₹8,499",4.4,"15,708 Ratings",https://rukminim1.flixcart.com/image/312/312/x...
20,"realme Narzo 30 Pro 5G (Blade Silver, 128 GB)","₹19,999",4.3,"28,540 Ratings",https://rukminim1.flixcart.com/image/312/312/k...
21,"APPLE iPhone 12 (Red, 128 GB)","₹57,999",4.6,"1,87,758 Ratings",https://rukminim1.flixcart.com/image/312/312/k...
22,"POCO X3 Pro (Steel Blue, 128 GB)","₹25,999",4.3,"56,276 Ratings",https://rukminim1.flixcart.com/image/312/312/k...


In [14]:
final_df.to_csv('flipkart')